In [1]:
import os

import numpy as np
import pandas as pd
from PIL import Image, ImageDraw


import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import datasets, models, transforms
from torchvision.models.detection.retinanet import RetinaNet
from torchvision.models.detection.faster_rcnn import FasterRCNN
import  torchvision.transforms.functional as F
from torchvision.models.detection.anchor_utils import AnchorGenerator


from functions import *
from functions_torch import *

In [2]:
params = {}
params['target_size']=(800,1333)
params['batch_size'] = 4
params['lr'] = 0.005
ds_root = '../../ladd-and-weights/dataset/pretrain/sdd-lacmus-version/'
weights_root = '../../ladd-and-weights/weights/torch/pretrain/'

In [3]:
class SDDDataSET(torch.utils.data.Dataset):
    def __init__(self, typeOfDS, transforms=None):
        self.typeOfDS = typeOfDS
        self.labels = pd.read_csv(os.path.join(ds_root,'%s_annotations_pedestrian.csv'%typeOfDS), header = None, 
            names = ['image','x0','y0','x1','y1','class'])
        self.transforms = transforms
    
    def __len__(self):
        return self.labels.image.nunique()

    def __getitem__(self, idx):
        # load images ad masks
        img_name = self.labels.image.unique()[idx]
        img_labels = self.labels [self.labels.image == img_name]
        img = Image.open(ds_root+img_name).convert("RGB") #convert from tutorial, do we need it?

        # get bounding box coordinates 
        num_objs = img_labels.shape[0]
        boxes = []
        for l in img_labels.iterrows():
            boxes.append([l[1]['x0'], l[1]['y0'], l[1]['x1'], l[1]['y1']])


        target = {}
        target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)         # there is only one class
        target["labels"] = labels = torch.ones((num_objs,), dtype=torch.int64)

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target


In [4]:
# Uncomment to test dataset implementation
# im_idx = 10

# dataset = SDDDataSET('test',get_transform(train=True)) 
# (image,target) = dataset[im_idx] 
# im = F.to_pil_image(image)
# draw = ImageDraw.Draw(im)

# for bb in target['boxes']:
#     draw.line([(bb[0], bb[1]), (bb[0], bb[3]), (bb[2], bb[3]),
#                (bb[2], bb[1]), (bb[0], bb[1])], width=4, fill=(255, 0, 0))

# im.show()

In [5]:
dataset_train = SDDDataSET('train', get_transform(train=True,target_size=params['target_size']))
dataset_val = SDDDataSET('val', get_transform(train=False,target_size=params['target_size']))

# split the dataset in train and test set
torch.manual_seed(1)

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset_train, batch_size=params['batch_size'], shuffle=True, num_workers=4
     ,collate_fn=collate_fn
)

data_loader_val = torch.utils.data.DataLoader(
    dataset_val, batch_size=1, shuffle=False, num_workers=4
     ,collate_fn=collate_fn
)




In [6]:
# #  Docstring (don't work)
# anchor_generator = AnchorGenerator(
#             sizes=((16, 32, 64, 128, 256),),
#             aspect_ratios=((0.25, 0.5, 1.0, 2.0, 4.0),), 
#          )

# # Retina anchors
# [anchor_parameters]
# sizes   = 16 32 64 128 256
# strides = 8 16 32 64 128
# ratios  = 0.5 1 2 3
# scales  = 1 1.2 1.6  # approx equal to 2**(1.0/3)=1.26 and 2**(2.0/3)=1.59

anchor_sizes = tuple((x, int(x * 2 ** (1.0 / 3)), int(x * 2 ** (2.0 / 3))) for x in [16, 32, 64, 128, 256])
aspect_ratios = ((0.5, 1.0, 2.0, 3.0),) * len(anchor_sizes)
anchor_generator = AnchorGenerator(
    anchor_sizes, aspect_ratios
)
# # load an instance segmentation model pre-trained on COCO    
model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=False, num_classes=2, pretrained_backbone=True,
                                                            min_size=params['target_size'][0], max_size = params['target_size'][1],
                                                           anchor_generator=anchor_generator)


In [7]:
# # load an instance segmentation model pre-trained on COCO    

# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, num_classes=2, pretrained_backbone=True,
#                                                             min_size=params['target_size'][0], max_size = params['target_size'][1] )

In [8]:

# the computation device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=params['lr'], momentum=0.9, weight_decay=0.0005) #lr 0.001 -> 0.005
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [9]:
for epoch in range(10):

    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=250)
    print ("Train done, evaluating.")
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    inference_res = evaluate(model,data_loader_val)
    print('Inference done, computing mAp : ')
    print(evaluate_res(inference_res, iou_threshold = 0.5, score_threshold = 0.05))    
    print(evaluate_res(inference_res, iou_threshold = 0.6, score_threshold = 0.05))
    print('Epoch Done')
    torch.save(model.state_dict(), 'resnet50_FRCNN_SDD_epoch_%i.pth'%epoch)

Epoch: [0]  [  0/639]  eta: 0:24:14  lr: 0.000013  loss: 1.8359 (1.8359)  classification: 1.1366 (1.1366)  bbox_regression: 0.6993 (0.6993)  time: 2.2770  data: 1.1841  max mem: 4054
Epoch: [0]  [250/639]  eta: 0:05:44  lr: 0.001970  loss: 1.6466 (1.7348)  classification: 1.1115 (1.1213)  bbox_regression: 0.5336 (0.6135)  time: 0.8860  data: 0.0148  max mem: 4355
Epoch: [0]  [500/639]  eta: 0:02:03  lr: 0.003927  loss: 1.5896 (1.6548)  classification: 1.0361 (1.0722)  bbox_regression: 0.5564 (0.5826)  time: 0.8861  data: 0.0149  max mem: 4368
Epoch: [0]  [638/639]  eta: 0:00:00  lr: 0.005000  loss: 1.4257 (1.6206)  classification: 0.8975 (1.0474)  bbox_regression: 0.5371 (0.5731)  time: 0.8859  data: 0.0158  max mem: 4416
Epoch: [0] Total time: 0:09:27 (0.8885 s / it)
Train done, evaluating.
Inference done, computing mAp : 
0.008726362038899503
0.0005682533818497805
Epoch Done
Epoch: [1]  [  0/639]  eta: 0:21:07  lr: 0.005000  loss: 1.3550 (1.3550)  classification: 0.8288 (0.8288)  bbo

In [ ]:
# # uncomment to test evaluation model and show detections

# cpu_device = torch.device("cpu")
# model.eval()
# for images, targets in data_loader:
#     g_images = list(img.to(device) for img in images)

#     if torch.cuda.is_available():
#         torch.cuda.synchronize()
#     outputs = model(g_images)

#     outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
#     res = targets, outputs
#     break

# im = F.to_pil_image(images[0])
# targets
# # im = to_pil_image(dataset[10][0])
# draw = ImageDraw.Draw(im)

# for bb in outputs[0]['boxes'][:10]:
#     draw.line([(bb[0], bb[1]), (bb[0], bb[3]), (bb[2], bb[3]),
#                (bb[2], bb[1]), (bb[0], bb[1])], width=4, fill=(255, 0, 0))

# for bb in targets[0]['boxes'][:10]:
#     draw.line([(bb[0], bb[1]), (bb[0], bb[3]), (bb[2], bb[3]),
#                (bb[2], bb[1]), (bb[0], bb[1])], width=4, fill=(0,255, 0))
# im.show()

# # # This suggests test set is far from perfect